# MNIST Prediction with Convolutional Neural Network

- MNIST dataset: is a dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. More info can be found at the [MNIST homepage](http://yann.lecun.com/exdb/mnist/).
- Goal: build a simple artificial neural network to predict the digit in the images.
- Reference: [Oddly Satisfying Deep Learning](https://pythonandml.github.io/dlbook/content/convolutional_neural_networks/cnn_over_mlp.html)

#### Import libraries

In [76]:
# for linear algebra
import numpy as np

# for plotting data, loss, accuracy
import matplotlib.pyplot as plt

# loading mnist dataset from keras
from keras import datasets

# show progress bar
from tqdm import tqdm

# for type hinting
from typing import Optional, Union

#### 1. Utils Functions

1. **plot_data**: plot the random 8 images from the dataset.
2. **Base Layer**: Base class for all the layers.
3. **Activation Functions**: Linear, reLU, Sigmoid, Tanh, Softmax.
3. **Weight Initialization**: Zeros, Ones, Random, Random Uniform.
4. **Optimization Functions**: Gradient Descent, Stochastic Gradient Descent, RMSprop, Adam.

##### 1.1. Plotting Functions

In [77]:
def plot_data(
    X: np.ndarray, y: np.ndarray, y_proba: Optional[np.ndarray] = None
) -> None:
    nrows, ncols = 2, 4
    _, axes = plt.subplots(nrows, ncols, figsize=(8, 4))

    len_x = X.shape[0]
    for idx in range(nrows * ncols):
        ax = axes[idx // ncols, idx % ncols]

        img_idx = np.random.randint(0, len_x)

        ax.imshow(X[img_idx], cmap="gray")
        ax.set(xticks=[], yticks=[])

        true_label = f"True: {y[img_idx]}"
        color = "black"

        if y_proba is not None:
            pred_label = f"Pred: {y_proba[img_idx]}"
            color = "green" if y[img_idx] == y_proba[img_idx] else "red"

        img_title = true_label if y_proba is None else f"{true_label}\n{pred_label}"
        ax.set_xlabel(img_title, color=color)

    plt.tight_layout()
    plt.show()

##### 1.2. Base Layer class

In [78]:
class BaseLayer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, X: np.ndarray) -> np.ndarray:
        """
        :param X: input data

        TODO: return output
        """
        pass

    def backpropagation(self, dZ: np.ndarray, lr: float) -> np.ndarray:
        """
        :param dZ: gradient of loss with respect to output
        :param lr: learning rate

        TODO: update parameters and return input gradient
        """
        pass

##### 1.2. Activation Functions class

In [79]:
class Activation(BaseLayer):
    def __init__(self, act: str = "linear") -> None:
        """
        :param act: activation function's name (relu, sigmoid, tanh, linear)
        """
        self.act = act

    def linear(self, x: np.ndarray) -> np.ndarray:
        return x

    def d_linear(self, x: np.ndarray) -> np.ndarray:
        return np.ones(x.shape)

    def reLU(self, x: np.ndarray) -> np.ndarray:
        return x * (x > 0)

    def d_reLU(self, x: np.ndarray) -> np.ndarray:
        return (x > 0) * np.ones(x.shape)

    def sigmoid(self, x: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-x))

    def d_sigmoid(self, x: np.ndarray) -> np.ndarray:
        return self.sigmoid(x) * (1 - self.sigmoid(x))

    def tanh(self, x: np.ndarray) -> np.ndarray:
        return np.tanh(x)

    def d_tanh(self, x: np.ndarray) -> np.ndarray:
        return 1 - self.tanh(x) ** 2

    def softmax(self, x: np.ndarray) -> np.ndarray:
        z = x - np.max(x, axis=-1, keepdims=True)
        numerator = np.exp(z)
        denominator = np.sum(numerator, axis=-1, keepdims=True)
        return numerator / denominator

    def d_softmax(self, x: np.ndarray) -> np.ndarray:
        if len(x.shape) == 1:
            x = np.array(x).reshape(1, -1)
        else:
            x = np.array(x)

        _, d = x.shape
        a = self.softmax(x)
        tensor1 = np.einsum("ij,ik->ijk", a, a)
        tensor2 = np.einsum("ij,jk->ijk", a, np.eye(d, d))

        return tensor2 - tensor1

    def get_activation(self, X: np.ndarray) -> np.ndarray:
        """
        :param X: input data to apply activation function
        """
        if self.act == "linear":
            return self.linear(X)
        elif self.act == "reLU":
            return self.reLU(X)
        elif self.act == "sigmoid":
            return self.sigmoid(X)
        elif self.act == "tanh":
            return self.tanh(X)
        elif self.act == "softmax":
            return self.softmax(X)
        else:
            raise ValueError(
                "Valid activation functions are linear, reLU, sigmoid, tanh, softmax"
            )

    def get_d_activation(self, X: np.ndarray) -> np.ndarray:
        if self.act == "linear":
            return self.d_linear(X)
        elif self.act == "reLU":
            return self.d_reLU(X)
        elif self.act == "sigmoid":
            return self.d_sigmoid(X)
        elif self.act == "tanh":
            return self.d_tanh(X)
        elif self.act == "softmax":
            return self.d_softmax(X)
        else:
            raise ValueError(
                "Valid activation functions are linear, reLU, sigmoid, tanh, softmax"
            )

    def forward(self, X: np.ndarray) -> np.ndarray:
        """
        :param X: input data to apply activation function
        """
        self.X = X
        return self.get_activation(X)

    def backpropagation(self, dZ: np.ndarray, lr: float) -> np.ndarray:
        """
        :param dZ: gradient of loss with respect to output
        :param lr: learning rate
        """
        f_prime = self.get_d_activation(self.X)

        if self.activation_type == "softmax":
            dx = np.einsum("ijk,ik->ij", f_prime, dZ)
        else:
            dx = dZ * f_prime
        return dx

##### 1.3. Weight Initialization class

- Zeros initialization: $w = np.zeros(shape)$
- Ones initialization: $w = np.ones(shape)$
- Random initialization: $w = np.random.randn(shape)$
- Random uniform initialization: $w = np.random.uniform(size=shape)$

In [80]:
class WeightInitializer:
    def __init__(self, shape, init: str = "random", seed: int = 69) -> None:
        """
        :param shape: shape of the weight matrix
        :param init: type of initialization (available initializations: zeros, ones, random, random_uniform)
        :param seed: seed for random initialization
        """
        self.shape = shape
        self.init = init
        self.seed = seed

    def zeros(self) -> np.ndarray:
        if self.seed is not None:
            np.random.seed(self.seed)
        return np.zeros(self.shape)

    def ones(self) -> np.ndarray:
        if self.seed is not None:
            np.random.seed(self.seed)
        return np.ones(self.shape)

    def random(self) -> np.ndarray:
        if self.seed is not None:
            np.random.seed(self.seed)
        return np.random.normal(size=self.shape)

    def random_uniform(self) -> np.ndarray:
        if self.seed is not None:
            np.random.seed(self.seed)
        return np.random.uniform(size=self.shape)

    def get_initializer(self) -> np.ndarray:
        if self.init == "zeros":
            return self.zeros()
        elif self.init == "ones":
            return self.ones()
        elif self.init == "random":
            return self.random()
        elif self.init == "random_uniform":
            return self.random_uniform()
        else:
            raise ValueError(
                "Valid initializations are: zeros, ones, random, random_uniform"
            )

##### 1.4.  Optimizers class

- Gradient Descent Optimizer: $w = w - \alpha \nabla_w L(w)$
- Stochastic Gradient Descent Optimizer: $w = w - \alpha \nabla_w L(w)$
- RMSprop Optimizer: $v = \beta v + (1 - \beta) \nabla_w L(w) \odot \nabla_w L(w)$ and $w = w - \alpha \frac{\nabla_w L(w)}{\sqrt{v + \epsilon}}$
- Adam Optimizer: $m = \beta_1 m + (1 - \beta_1) \nabla_w L(w)$, $v = \beta_2 v + (1 - \beta_2) \nabla_w L(w) \odot \nabla_w L(w)$, $m_{\text{corrected}} = \frac{m}{1 - \beta_1^t}$, $v_{\text{corrected}} = \frac{v}{1 - \beta_2^t}$, and $w = w - \alpha \frac{m_{\text{corrected}}}{\sqrt{v_{\text{corrected}} + \epsilon}}$

> Note: Actually, i only use the Gradient Descent Optimizer in this notebook.

In [81]:
class Optimizer:
    def __init__(
        self,
        op_type: str = "GD",
        shape_W: tuple[int, int] = None,
        shape_b: tuple[int, int] = None,
        m1: float = 0.9,
        m2: float = 0.999,
        epsilon: int = 1e-8,
    ) -> None:
        """
        :param op_type: type of optimizer (available optimizers: GD, SGD, RMSProp, Adam)
        :param shape_W: shape of the weight matrix
        :param shape_b: shape of the bias matrix
        :param m1: hyperparameter >= 0 that accelerates gradient descent in the relevant direction and dampens oscillations. Used in RMSprop
        :param m2: hyperparameter for adam only
        :param epsilon: parameter used in adam and RMSprop to prevent division by zero error
        """
        self.op_type = op_type
        self.m1 = m1
        self.m2 = m2
        self.epsilon = epsilon

        self.vdW = np.zeros(shape_W)
        self.vdb = np.zeros(shape_b)

        self.SdW = np.zeros(shape_W)
        self.Sdb = np.zeros(shape_b)

    def GD(self, dW: np.ndarray, db: np.ndarray, _: int) -> tuple:
        """
        :param dW: gradient of Weight W for iteration k
        :param db: gradient of bias b for iteration k
        :param _: iteration number
        """
        return dW, db

    def SGD(self, dW: np.ndarray, db: np.ndarray, _: int) -> tuple:
        """
        :param dW: gradient of Weight W for iteration k
        :param db: gradient of bias b for iteration k
        :param _: iteration number
        """
        self.vdW = self.m1 * self.vdW + (1 - self.m1) * dW
        self.vdb = self.m1 * self.vdb + (1 - self.m1) * db

        return self.vdW, self.vdb

    def RMSProp(self, dW: np.ndarray, db: np.ndarray, _: int) -> tuple:
        """
        :param dW: gradient of Weight W for iteration k
        :param db: gradient of bias b for iteration k
        :param k: iteration number
        """
        self.SdW = self.m2 * self.SdW + (1 - self.m2) * (dW**2)
        self.Sdb = self.m2 * self.Sdb + (1 - self.m2) * (db**2)

        den_W = np.sqrt(self.SdW) + self.epsilon
        den_b = np.sqrt(self.Sdb) + self.epsilon

        return dW / den_W, db / den_b

    def Adam(self, dW: np.ndarray, db: np.ndarray, k: int) -> tuple:
        """
        :param dW: gradient of Weight W for iteration k
        :param db: gradient of bias b for iteration k
        :param k: iteration number
        """
        # momentum
        self.vdW = self.m1 * self.vdW + (1 - self.m1) * dW
        self.vdb = self.m1 * self.vdb + (1 - self.m1) * db

        # rmsprop
        self.SdW = self.m2 * self.SdW + (1 - self.m2) * (dW**2)
        self.Sdb = self.m2 * self.Sdb + (1 - self.m2) * (db**2)

        # correction
        if k > 1:
            vdW_h = self.vdW / (1 - (self.m1**k))
            vdb_h = self.vdb / (1 - (self.m1**k))
            SdW_h = self.SdW / (1 - (self.m2**k))
            Sdb_h = self.Sdb / (1 - (self.m2**k))
        else:
            vdW_h = self.vdW
            vdb_h = self.vdb
            SdW_h = self.SdW
            Sdb_h = self.Sdb

        den_W = np.sqrt(SdW_h) + self.epsilon
        den_b = np.sqrt(Sdb_h) + self.epsilon

        return vdW_h / den_W, vdb_h / den_b

    def get_optimizer(self, dW: np.ndarray, db: np.ndarray, k: int) -> tuple:
        """
        :param dW: gradient of Weight W for iteration k
        :param db: gradient of bias b for iteration k
        :param k: iteration number
        """
        if self.op_type == "GD":
            return self.GD(dW, db, k)
        elif self.op_type == "SGD":
            return self.SGD(dW, db, k)
        elif self.op_type == "RMSProp":
            return self.RMSProp(dW, db, k)
        elif self.op_type == "Adam":
            return self.Adam(dW, db, k)
        else:
            raise ValueError("Valid optiomizers are GD, SGD, RMSProp, Adam")

##### 1.5. Loss Functions class

- Mean Squared Error Loss: $L(y, \hat{y}) = \frac{1}{2} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$
- Derivative of Mean Squared Error Loss: $\frac{\partial L(y, \hat{y})}{\partial \hat{y}} = \hat{y} - y$
- Binary Cross Entropy Loss: $L(y, \hat{y}) = - \sum_{i=1}^{n} y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)$
- Derivative of Binary Cross Entropy Loss: $\frac{\partial L(y, \hat{y})}{\partial \hat{y}} = - \frac{y}{\hat{y}} + \frac{1 - y}{1 - \hat{y}$

In [82]:
class Loss:
    def __init__(self, loss: str = "mse") -> None:
        """
        :param loss: str, loss function (Available: mse, cross-entropy)
        """
        self.loss = loss

    # Mean Squared Error
    def mse(self, a: np.ndarray, y: np.ndarray) -> float:
        """
        :param a: predicted value
        :param y: true value
        """
        return (1 / 2) * np.sum((np.linalg.norm(a - y, axis=1)) ** 2)

    def d_mse(self, a: np.ndarray, y: np.ndarray) -> float:
        """
        :param a: predicted value
        :param y: true value
        """
        return a - y

    # Binary Cross Entropy
    def cross_entropy(
        self, a: np.ndarray, y: np.ndarray, epsilon: float = 1e-12
    ) -> float:
        """
        :param a: predicted value
        :param y: true value
        """
        a = np.clip(a, epsilon, 1.0 - epsilon)
        return -np.sum(y * np.log(a))

    def d_cross_entropy(
        self, a: np.ndarray, y: np.ndarray, epsilon: float = 1e-12
    ) -> float:
        """
        :param a: predicted value
        :param y: true value
        """
        a = np.clip(a, epsilon, 1.0 - epsilon)
        return -y / a

    def get_loss(self, a: np.ndarray, y: np.ndarray) -> float:
        """
        :param a: predicted value
        :param y: true value
        """
        if self.loss == "mse":
            return self.mse(a, y)
        elif self.loss == "cross-entropy":
            return self.cross_entropy(a, y)
        else:
            raise ValueError("Valid losses are mse, cross-entropy")

    def get_d_loss(self, a: np.ndarray, y: np.ndarray) -> float:
        """
        :param a: predicted value
        :param y: true value
        """
        if self.loss == "mse":
            return self.d_mse(a, y)
        elif self.loss == "bce":
            return self.d_bce()
        else:
            raise ValueError("Valid losses are mse, cross-entropy")

##### 1.6. Learning Rate Decay class

In [83]:
class LearningRateDecay:
    def __init__(self) -> None:
        pass

    def constant(self, t: int, lr: float) -> float:
        """
        :param t: iteration number
        :param lr: learning rate initial value
        """
        return lr

    def time_decay(self, t: int, lr: float, k: float) -> float:
        """
        :param t: iteration number
        :param lr: learning rate initial value
        :param k: decay rate
        """
        return lr / (1 + (k * t))

    def step_decay(self, t: int, lr: float, F: int, D: float) -> float:
        """
        :param t: iteration number
        :param lr: learning rate initial value
        :param F: fractor value controlling the decay
        :param D: "Drop every" iteration
        """
        return lr * (F ** np.floor((1 + t) / D))

    def exponential_decay(self, t: int, lr: float, k: float) -> float:
        """
        :param t: iteration number
        :param lr: learning rate initial value
        :param k: decay rate
        """
        return lr * np.exp(-k * t)

#### 2. Convolutional Neural Network

##### 2.1. Convolutional Layer

In [84]:
class Padding2D(BaseLayer):
    def __init__(self, p: Union[str, tuple[int, int]] = "valid") -> None:
        """
        :param p: padding type (valid, same or tuple[int,int])
        """
        self.p = p

    def get_dimensions(
        self, inp_shape: tuple[int, int], kernel_size: int, s: tuple[int, int] = (1, 1)
    ) -> tuple:
        """
        :param inp_shape: input shape (H,W)
        :param kernel_size: kernel size
        :param s: stride
        """
        if len(inp_shape) == 4:
            m, Nc, Nh, Nw = inp_shape
        elif len(inp_shape) == 3:
            Nc, Nh, Nw = inp_shape

        Kh, Kw = kernel_size
        Sh, Sw = s
        p = self.p

        if type(p) == int:
            pt, pb = p, p
            pl, pr = p, p
        elif type(p) == tuple:
            ph, pw = p
            pt, pb = ph // 2, (ph + 1) // 2
            pl, pr = pw // 2, (pw + 1) // 2
        elif p == "valid":
            pt, pb = 0, 0
            pl, pr = 0, 0
        elif p == "same":
            # calculating how much padding is required in all 4 directions
            # (top, bottom, left and right)
            ph = (Sh - 1) * Nh + Kh - Sh
            pw = (Sw - 1) * Nw + Kw - Sw

            pt, pb = ph // 2, (ph + 1) // 2
            pl, pr = pw // 2, (pw + 1) // 2

        else:
            raise ValueError("Valid padding types are: valid, same or tuple")

        if len(inp_shape) == 4:
            out_shape = (m, Nc, Nh + pt + pb, Nw + pl + pr)
        elif len(inp_shape) == 3:
            out_shape = (Nc, Nh + pt + pb, Nw + pl + pr)

        return out_shape, (pt, pb, pl, pr)

    def forward(
        self, X: np.ndarray, kernel_size: int, s: tuple[int, int] = (1, 1)
    ) -> np.ndarray:
        """
        :param X: input data
        :param kernel_size: kernel size
        :param s: stride

        :return X_pad: padded input data
        """

        self.inp_shape = X.shape
        m, Nc, Nh, Nw = X.shape

        self.out_shape, (self.pt, self.pb, self.pl, self.pr) = self.get_dimensions(
            self.inp_shape, kernel_size, s
        )

        zeros_r = np.zeros((m, Nc, Nh, self.pr))
        zeros_l = np.zeros((m, Nc, Nh, self.pl))
        zeros_t = np.zeros((m, Nc, self.pt, Nw + self.pl + self.pr))
        zeros_b = np.zeros((m, Nc, self.pb, Nw + self.pl + self.pr))

        X_pad = np.concatenate((X, zeros_r), axis=3)
        X_pad = np.concatenate((zeros_l, X_pad), axis=3)
        X_pad = np.concatenate((zeros_t, X_pad), axis=2)
        X_pad = np.concatenate((X_pad, zeros_b), axis=2)

        return X_pad

    def backpropagation(self, dZ: np.ndarray, lr: float) -> np.ndarray:
        """
        :param dZ: Backprop Error of padded X (Xp)

        :return dX: Backprop Error of X
        """
        m, Nc, Nh, Nw = self.inp_shape
        dX = dZ[:, :, self.pt : self.pt + Nh, self.pl : self.pl + Nw]
        return dX